Наброски кода для
0. сведения алломорфов морфем в одно (хотя бы в таблице)
1. подсчёта сочетаний морфем
2. вытаскивания конвербов
3. вытаскивания цепочек частей речи

Все манипуляции можно тестировать на корпусах, которые в почте.

# 0
Действительно надо вытащить всё, положить в словарину и радоваться

Формат:

`корпус{
    название:документ{    # done
        мета:мета,    # done
        текст:[
            предложение{
                слой:[
                    морфемы
                    ],
                перевод:''
            }
        ]
    }
}`

In [2]:
corpora = ['Kamchatka', 'Sebjan']

In [66]:
import os, re, pickle
from pprint import pprint


def morphs_2_words(line):
    '''берёт на вход массив расчленённой строки, возвращает массив слов'''
    words = []
    word = []
    for morph in line:
        if morph.strip()[0] in '=-':
            word.append(morph)
        else:
            if len(word) > 0:
                words.append(word)
            word = [morph]
    words.append(word)
    return words


def lines_2_dict(part):
    '''
    i: кусок текста (предложение) в несколько строк, в каждой строке несколько слоёв, и с другими данными предложения
    o: джейсонина вида {'слой': [сл, о, ва], 'слой': содержимое}
    доп. ограничения: длина всех строк-массивов равна
    '''
    res = {}
    # собирать по строке в словарь
    # mad
    lines = [line for line in part.split('\n')[1:] if len(line) > 1] # leaving out the first line with index
    parted_layers = ['tx', 'mb', 'ge']
    for line in lines:
        line = line.split()
        layer = line[0].strip('\\')
        if layer in parted_layers:
            line_content = morphs_2_words(line[1:])
        elif layer == 'ps':
            line_content = line[1:]
        else:
            line_content = ' '.join(line[1:])
        if layer in res:
            res[layer] += line_content
        else:
            res[layer] = line_content
    return res


def make_readable(corp):
    '''
    переводит текст корпусов в удобомашиночитаемую джейсонину.
    '''
    folder = 'Corpus_Text_{}_postagged'.format(corp)
    corpus_dict = {}
    for fil in os.listdir(folder):
        if not fil.endswith('.txt'):
            continue
        with open(os.path.join(folder, fil), 'r') as f:
            text = f.read()
        file_content = {}
        sents = text.split('\id')
        file_content['meta'] = sents[1] # metainfo at the beginning of the file; not parsed
        text = []
        for sent in sents[2:]:
#             parts = re.findall('\\tx.*?\n\n', text, flags=re.DOTALL)
            sent_content = lines_2_dict(sent)
            text.append(sent_content)
        file_content['text'] = text
        corpus_dict[fil] = file_content
    with open('{}.pickle'.format(corp), 'wb') as f:
        pickle.dump(corpus_dict, f)
    return corpus_dict

In [70]:
def detect(string, layer, corpora=corpora, show=True):
    for corpus in corpora:
        hits = []
        with open('{}.pickle'.format(corpus), 'rb') as f:
            content = pickle.load(f)
        for doc in content:
            for i in range(len(content[doc]['text'])): # looping through sentences
                if not layer in content[doc]['text'][i]:
                    continue
                if string in content[doc]['text'][i][layer]:
                    print('Found in {}, {} at {}'.format(corpus, doc, i))
                    if show:
                        pprint(content[doc]['text'][i])
                    hits.append(doc)
        if len(hits) == 0:
            print('noth found. Cool!')
        else:
            inp = input('correct all these ({} hits) & press Enter and I\'ll reload.\n'.format(len(hits)))
            if inp == '\n':
                make_readable(corpus)
            elif 'replace' in inp:
                target = inp.split()[1]
                for doc in list(set(hits)):
                    with open(os.path.join('Corpus_Text_{}_postagged'.format(corpus), doc), 'r') as f:
                        text = f.read()
                    text = re.sub(string, target, text)
                    with open(os.path.join('Corpus_Text_{}_postagged'.format(corpus), doc), 'w') as f:
                        f.write(text)
            print('done!')

In [67]:
for corp in corpora:
    make_readable(corp)

In [69]:
detect('excl', 'ps')

Found in Kamchatka, Ichanga_Adukanov_muzej2.txt at 41
['excl', 'n', 'ptl']
Found in Kamchatka, Ichanga_Adukanov_muzej2.txt at 77
['pron', 'ptl', 'adj', 'excl', 'pron', 'adv', 'pron']
Found in Kamchatka, Tylkanova_Lidija_Gavrilevna_2.txt at 98
['pron', 'n', 'v', 'pron', 'adj', 'v', 'n', 'excl']
Found in Kamchatka, Egorova_RM_Anin_reincarn_soul_1.txt at 38
['excl', 'interj']
Found in Kamchatka, Banakanova_PM_childhood_olenevod.txt at 20
['conj', 'n', 'n', 'v', 'v', 'excl', 'v', 'adj', 'n', '?', 'pron', '?', 'v']
Found in Kamchatka, Berezina_IN_nimkan_NA.txt at 28
['adv', 'pron', 'excl', 'excl', 'v', 'excl', 'pron', 'pron']
Found in Kamchatka, Ichanga_Adukanov_muzej1_cont.txt at 216
['excl', 'n', 'pron']
Found in Kamchatka, Ichanga_Adukanov_muzej1_cont.txt at 241
['excl', 'n']
Found in Kamchatka, Ichanga_Adukanov_muzej1_cont.txt at 264
['excl']
Found in Kamchatka, Ichanga_Adukanov_muzej1_cont.txt at 287
['excl', 'n', 'n', 'interj', 'n', 'v']
Found in Kamchatka, Ichanga_Adukanov_muzej1_con

In [63]:
with open('{}.pickle'.format('Kamchatka'), 'rb') as f:
    content = pickle.load(f)
pprint(content['Egorova_RM_Utakan.txt']['text'][22]['ps'])

[['v'],
 ['n'],
 ['ideophone'],
 ['ideophone'],
 ['n'],
 ['ideophone'],
 ['ideophone']]


# 1
это уже начато там
морфемы лежат в отдельном словаре, кажется; нужно приложить это знание на таблицу?
или создать уже наконец промежуточное представление всего

# 2
сочетания морфем

сначала надо вытащить представления (кажется это уже сделано)

пройтись по всему, растащить каждое слово на пары морфем, первая (ROOT, morph), последняя (morph, END)

`from collections import Counter
# вытаскиваю все морфемы из первого слота, считаю, с чем они сочитаются, записываю в словарь
morph_pairs_dict = {x: Counter([pair for pair in morph_pairs_list if pair[0]==x]) for x in set(map(lambda pair: pair[0], morph_pairs_list))}
`
